In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install torch torchvision torchaudio

In [3]:
!pip install Cython

In [4]:
!pip install nemo_toolkit['all']

In [5]:
# !wget -O checkpoint.ckpt "https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020205_vnu_edu_vn/EbYkGgMABVBFg3oiM9iWA3MBnjmxvVIJJ-5BEDUwJ7w7LQ?e=uwmuCM&download=1"

In [6]:
!wget -O checkpoint.ckpt "https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020108_vnu_edu_vn/Ef7m8t9hPMtPkkmP53dFnpEBXhFsC6HzOIO3bjD_actStw?e=ThGyC6&download=1"

--2023-12-17 18:38:43--  https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020108_vnu_edu_vn/Ef7m8t9hPMtPkkmP53dFnpEBXhFsC6HzOIO3bjD_actStw?e=ThGyC6&download=1
Resolving vnueduvn-my.sharepoint.com (vnueduvn-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to vnueduvn-my.sharepoint.com (vnueduvn-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/21020108_vnu_edu_vn/Documents/checkpoint.ckpt?ga=1 [following]
--2023-12-17 18:38:44--  https://vnueduvn-my.sharepoint.com/personal/21020108_vnu_edu_vn/Documents/checkpoint.ckpt?ga=1
Reusing existing connection to vnueduvn-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 152292670 (145M) [application/octet-stream]
Saving to: ‘checkpoint.ckpt’

checkpoint.ckpt     100%[===================>] 145.24M  40.1MB/s    in 6.7s    

2023-12-17 18:38:51 (21.5 MB/s) - ‘checkpoint.ckpt’ saved [152292670/152292670]



In [7]:
import os
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML

# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import DictConfig
import copy
import pytorch_lightning as pl
import torch

In [19]:
class RunModel:
    def __init__(self) -> None:
        self.model = nemo_asr.models.EncDecCTCModel.load_from_checkpoint("checkpoint.ckpt", map_location='cpu')

    def get_result(self, path2audio_files) :
        batch_sz = 4
        trans = []
        batch = []
        for i in range(len(path2audio_files)) :
            batch.append(path2audio_files[i])
            if (i % batch_sz == batch_sz - 1) :
                rt = self.model.transcribe(paths2audio_files=batch, batch_size=len(batch))
                for _ in rt:
                    trans.append(_)
                batch = []
        rt = self.model.transcribe(paths2audio_files=batch, batch_size=len(batch))
        for _ in rt:
            trans.append(_)
        # return model.transcribe(paths2audio_files=path2audio_files, batch_size=len(path2audio_files))
        return trans

model = RunModel()

[NeMo W 2023-12-17 18:45:44 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /work/hpc/potato/asr/data/LibriSpeech/dev-clean_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-12-17 18:45:44 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_mult

[NeMo I 2023-12-17 18:45:44 features:289] PADDING: 16


In [9]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install flask[async]
!ngrok authtoken 2ZDpxzhxZP7h0QMThOw4FJkJrIS_3DCv44LHYHKX1jNNMzVn8

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
!apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [11]:
# !pip install aiohttp[speedups]
# !pip install nest_asyncio

In [12]:
import soundfile as sf
import numpy as np
import librosa

In [27]:
def split_wav(input_path, output_folder, segment_duration=10):
    y, sr = librosa.load(input_path, sr=None)

    segment_frames = int(segment_duration * sr)
    files = []

    for i in range(0, len(y), segment_frames):
        segment = y[i:i+segment_frames]
        output_file = f"{output_folder}/segment_{i // segment_frames + 1}.wav"
        sf.write(file=output_file, data=segment, samplerate=sr)
        files.append(output_file)
    return files

In [14]:
!mkdir divided

mkdir: cannot create directory ‘divided’: File exists


In [49]:
# input_path = "/content/xltn_team6/test10p.wav"
# output_folder = "/content/divided3"
# split_wav(input_path, output_folder)

def solve_a_file(input_path):
    split = split_wav(input_path, "divided")
    ans = model.get_result(split)
    rt = ""
    for x in ans:
        rt = rt + x + " "
    return (rt, input_path)

In [42]:
def split_many_file(input_paths):
    paths = []
    for p in input_paths:
        p.save(p.filename)
        path = solve_a_file(p.filename)
        paths.append(path)
    return paths

In [17]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
import subprocess
from io import StringIO

In [48]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
async def home():
    return """
        <h1>Please choose the audio file:</h1>
        <h3>Allowed audio file types: .wav</h3>
        <form action="/uploader" method="POST" enctype = "multipart/form-data">
            <input type = "file" name = "file" multiple/>
            <br>
            <input type = "submit"/>
        </form>
        <br>
        <p id="return"></p>
        """

# def process_file(files) :
#       # files = split_many_file(files)
#       path = []
#       duration = []
#       for file in files:
#           parts = file.filename.split(".")
#           if parts[-1] != "wav":
#               return "Not wav file"
#           local_filename = "flask_" + file.filename
#           print(local_filename)
#           file.save(local_filename)
#           path.append("/content/" + local_filename)
#           output = float(subprocess.check_output(['sox', '--i', '-D', local_filename]))
#           duration.append(output)
#       output = model.get_result(path)
#       ans = 'file uploaded successfully <br>'
#       for idx in range(len(output)):
#           ans = ans + "{text: " +  output[idx] + ", duration: " + str(duration[idx]) + "}" +  "<br>"
#       return ans

def process_file(files) :
      files = split_many_file(files)
      filename = []
      output = []
      duration = []
      for (plot, fi) in files:
          # filename = file.split('/')
          filename.append(fi)
          output.append(plot)
          durex = float(subprocess.check_output(['sox', '--i', '-D', fi]))
          duration.append(durex)
      # output = model.get_result(files)
      # print(len(output))
      ans = 'file uploaded successfully <br>'
      for idx in range(len(output)):
          ans = ans + "<li> file: " + filename[idx] + ", text: " +  output[idx] + ", duration: " + str(duration[idx]) + "s. </li>" +  "<br>"
      return ans

import time

@app.route('/uploader', methods = ['GET', 'POST'])
async def upload_file():
   if request.method == 'POST':
      # async with aiohttp.ClientSession("/") as session:
      #     async with session.get('/') as resp:
      #         print(resp.status)
      #         print(await resp.text())
      start_time = time.time()
      f = request.files.getlist('file')
      ans = process_file(f)
      duration = time.time() - start_time
      ans = ans + "<br>Duration: " + str(duration) + "s. <br><a href='/'>Back</a>"
      return ans

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://936a-35-196-83-65.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 19:20:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 19:20:50] "GET /favicon.ico HTTP/1.1" 404 -


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 19:23:57] "POST /uploader HTTP/1.1" 200 -


Note:
- limit batch
- ?